In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from utils import *

# Soft voting

In [ ]:
def mul(lst):
    return np.asarray(eval(lst))

def tolist(txt):
    return txt.tolist()

In [ ]:
k = int(input()) # 앙상블 모델 개수 입력
s = nn.Softmax()

mod_pow = torch.tensor(list(map(float, input().split()))) # score 입력
mod_pow = s(mod_pow)

tmp = pd.DataFrame()
output = pd.DataFrame()
for i in range(k):
    path = input().strip() # score csv 파일 경로 입력
    data = pd.read_csv(path) 
    tmp[i] = data['probs'].apply(mul) * float(mod_pow[i])
output['id'] = data['id']
output['probs'] = np.sum(tmp, axis=1)
pred = output['probs'].apply(np.argmax).tolist()
pred_a = num_to_label(pred)
output["pred_label"] = pred_a
output['probs'] = output['probs'].apply(tolist)
output

In [ ]:
output.to_csv('4e_submission.csv', index=False)

# Hard voting

In [ ]:
roberta_df = pd.read_csv('/opt/ml/code/level2_klue_nlp-level1-nlp-12/pl/ensemble/result/roberta_ensemble_submission.csv')
electra_df = pd.read_csv('/opt/ml/code/level2_klue_nlp-level1-nlp-12/pl/ensemble/result/electra_ensemble_test_submission3.csv')
r_roberta_df = pd.read_csv('/opt/ml/code/level2_klue_nlp-level1-nlp-12/pl/ensemble/result/r_roberta_submission.csv')
electra_roberta_r_roberta_df = pd.read_csv('/opt/ml/code/level2_klue_nlp-level1-nlp-12/pl/ensemble/result/electra_roberta_r_roberta_submission.csv')
subm_df = pd.read_csv('/opt/ml/code/level2_klue_nlp-level1-nlp-12/pl/ensemble/subm.csv')
bigbird_df = pd.read_csv('/opt/ml/code/level2_klue_nlp-level1-nlp-12/pl/ensemble/bibbird_cr_cs_od_submission(83.46).csv')
bigbird_df2 = pd.read_csv('/opt/ml/code/level2_klue_nlp-level1-nlp-12/pl/ensemble/bigbird_ls_cs_od_submission(83.14).csv')

In [ ]:
hard = pd.concat([roberta_df['pred_label'], 
                  electra_df['pred_label'],
                  r_roberta_df['pred_label'], 
                  electra_roberta_r_roberta_df['pred_label'],
                  subm_df['pred_label'], 
                  bigbird_df['pred_label'], 
                  bigbird_df['pred_label']], 
                 axis=1).mode(axis=1).iloc[:,0] # pred_label 최빈값

e_r_rr_s_bb = pd.concat([electra_roberta_r_roberta_df.iloc[:,:2],hard], axis=1) 
e_r_rr_s_bb.columns = ['id', 'probs', 'pred_label']
e_r_rr_s_bb

In [ ]:
e_r_rr_s_bb.to_csv('e_r_rr_s_bb_hard_submission.csv', index=False)